# LAMMPS Tutorials 05. Generate the $\Sigma5(310)$ Grain Boundary!

### Author: Mark Tschopp, mark.a.tschopp.civ at mail.mil

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/mrkllntschpp/lammps-tutorials.

The original tutorials are given here: https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_tutorials.  A number of these tutorials are out of date and have been ported over into the current iPython Jupyter Notebook tutorials on github.

***
## Abstract:
<a id="Sec1"></a>

The objective of this tutorial is to generate a symmetric tilt grain boundary in LAMMPS [1]. This tutorial can serve as a precursor to more advanced techniques, whereby in-plane translations and atom deletion criteria are used to sample a large number of potential structures to find the global minimum energy grain boundary structure [2-3].

</br>
<table width="900" border="0" cellpadding="5">
<tr>

<td width="250" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/d/d3/Sig5_1.jpg" width="300" title="Minimization of Sigma5 grain boundary">
</figure>
</td>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/1/1c/Al_sig5_1.gif" width="300" title="Minimization of Sigma5 grain boundary">
</figure>
</td>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/e/e8/Al_sig5_2.gif" width="300" title="Minimization of Sigma5 grain boundary (with different starting position)">
</figure>
</td>

</tr>
</table>

***
## Complete the First Tutorials

If you have not done so already, you may want to complete the first four tutorials available [here](https://github.com/mrkllntschpp/lammps-tutorials). 

***
## Description of Simulation 
This LAMMPS [1] molecular dynamics simulation first generates a simulation cell with fcc atoms with two blocks of differing orientations to create the grain boundary: a $<031>$ direction normal to the interface plane (often called grain boundary normal), a $<013>$ direction in the grain boundary plane (y-direction), a shared $<100>$ tilt direction in the z direction. This is a symmetric tilt grain boundary that is known as the $\Sigma5(310)$ symmetric tilt grain boundary, since both lattices are rotated about a common tilt direction, the $<100>$ direction.  The potential used is the Mishin *et al.* (1999) aluminum potential [4]. The cfg dump files include the x, y, and z coordinates, the centrosymmetry values, the potential energies, and forces for each atom. This can be directly visualized using OVITO [5] and AtomEye [6] visualization tools. 

## Step 1: Create the LAMMPS input script 
This input script was run using the January 2020 version of LAMMPS. The following input script shows how multiple translations and an atom deletion criteria are used to calculate the minimum energy structure. This input script for LAMMPS [1] can be called with a command of the form, `lmp_exe < input.script`. This script contains loops over x-translations, z-translations, and atom overlap distances (an atom is deleted when an atom pair with a nearest neighbor distance is less than this distance). The unique minimum energy structures are saved as a dump file with the energy appended to the filename in a new folder specified by the `gbname` variable. The dump files can then be easily scanned through for the global minimum energy structure. 

In [1]:
%%writefile in.al_sig5_310_stgb.txt

# LAMMPS Input File for Grain Boundaries 
# Mark Tschopp
# This file will generate a single Sigma5(310) STGB
# Syntax "lmp_exe < in.al_sig5_310_stgb.txt" from command prompt


# ---------- Initialize Simulation --------------------- 
clear 
units metal 
dimension 3 
boundary p p p 
atom_style atomic 

# ---------- Create Atomistic Structure --------------------- 
lattice fcc 4.05 
region whole block 0.000000 12.807225 -64.0361225 64.0361225 0.000000 4.050000 units box 
create_box 2 whole 
region upper block INF INF 0.000000 64.0361225 INF INF units box 
lattice fcc 4.05 orient x  0  3  1 orient y  0 -1  3 orient z  1  0  0 
create_atoms 1 region upper 
region lower block INF INF -64.0361225 0.000000 INF INF units box 
lattice fcc 4.05 orient x  0  3 -1 orient y  0  1  3 orient z  1  0  0 
create_atoms 2 region lower 
group upper type 1 
group lower type 2  

# ---------- Define Interatomic Potential --------------------- 
pair_style eam/alloy 
pair_coeff * * Al99.eam.alloy Al Al
neighbor 2.0 bin 
neigh_modify delay 10 check yes 
 
# ---------- Displace atoms and delete overlapping atoms --------------------- 
displace_atoms upper move 0 0 0 units lattice 
delete_atoms overlap 0.35 lower upper
 
# ---------- Define Settings --------------------- 
compute csym all centro/atom fcc
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

# ---------- Run Minimization --------------------- 
reset_timestep 0 
thermo 10 
thermo_style custom step pe lx ly lz press pxx pyy pzz c_eatoms 
dump 1 all cfg 25 dump.sig5_minimization_*.cfg mass type xs ys zs c_csym c_eng fx fy fz
dump_modify 1 element Al Al
min_style cg 
minimize 1e-15 1e-15 5000 5000 
undump 1

# ---------- Run Minimization 2--------------------- 
# Now allow the box to expand/contract perpendicular to the grain boundary
reset_timestep 0 
thermo 10 
thermo_style custom step pe lx ly lz press pxx pyy pzz c_eatoms 
fix 1 all box/relax y 0 vmax 0.001
min_style cg 
minimize 1e-15 1e-15 5000 5000 

# ---------- Calculate GB Energy --------------------- 
variable minimumenergy equal -3.360000
variable esum equal "v_minimumenergy * count(all)" 
variable xseng equal "c_eatoms - (v_minimumenergy * count(all))" 
variable gbarea equal "lx * lz * 2" 
variable gbe equal "(c_eatoms - (v_minimumenergy * count(all)))/v_gbarea" 
variable gbemJm2 equal ${gbe}*16021.7733 
variable gbernd equal round(${gbemJm2}) 
print "GB energy is ${gbemJm2} mJ/m^2" 
 
# ---------- Dump data into Data file ------------- 
reset_timestep 0 
dump 1 all cfg 10000 dump.al_sig5_310_*.cfg mass type xs ys zs c_csym c_eng fx fy fz
dump_modify 1 element Al Al
minimize 1e-15 1e-15 5000 5000
undump 1

write_restart restart.al_sig5_310_stgb

print "All done" 


Overwriting in.al_sig5_310_stgb.txt


So far so good. You should have the file in your working directory if you are running this from Jupyter Notebook.

## Step 2: Run the LAMMPS Simulation

Now run the simulation as we have done before.  On my computer, the 24Jan2020 LAMMPS executable is stored in the `C:\Program Files\LAMMPS 64-bit 24Jan2020\bin\` folder and is named `lmp_serial.exe`.  The `log.lammps` file should look like the output below. 

In [2]:
!"C:/Program Files/LAMMPS 64-bit 24Jan2020/bin/lmp_serial" < in.al_sig5_310_stgb.txt

LAMMPS (24 Jan 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (../comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.05 4.05 4.05
Created orthogonal box = (0 -64.0361 0) to (12.8072 64.0361 4.05)
  1 by 1 by 1 MPI processor grid
Lattice spacing in x,y,z = 5.12289 5.12289 4.05
Created 200 atoms
  create_atoms CPU = 0 secs
Lattice spacing in x,y,z = 5.12289 5.12289 4.05
Created 200 atoms
  create_atoms CPU = 0 secs
200 atoms in group upper
200 atoms in group lower
Displacing atoms ...
System init for delete_atoms ...
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost atom cutoff = 8.28721
  binsize = 4.1436, bins = 4 31 1
  2 neighbor lists, perpetual/occasional/extra = 1 1 0
  (1) command delete_atoms

The grain boundary energy is calculated as 567.57 mJ/m$^2$. The final configuration is stored in a restart file for future use in predicting properties of grain boundaries. Additionally, files with atomic coordinates, energies, and centrosymmetry values are dumped for post-processing. 

***
## Step 3: Making the Deformation Movie 

Not much postprocessing to do with this example, so try making a few movies like explained in  [Tutorial 3](LAMMPS-Tutorials-03.ipynb) and [Tutorial 4](LAMMPS-Tutorials-04.ipynb).

Figure 1 shows a movie of minimization of the grain boundary structure for the input script show above. To see the effect of changing the starting position, Figure 2 shows a movie of minimization of the grain boundary structure for a modification of the input script above. Here, the 

</br>
<div class="alert alert-block alert-info">
delete_atoms overlap 0.35 lower upper
</div>

line in the input script has been modified to 

</br>
<div class="alert alert-block alert-info">
delete_atoms overlap 1.5 lower upper
</div>

This eliminates two atoms that are too close to each other near the grain boundary interface (shown in Figure 4).  Interestingly, this simple example shows that the grain boundary energy landscape can be very complex due to a number of different energy minima, which can only be accessed through broadly sampling the grain boundary design space.

</br>
<table width="800" border="0" cellpadding="5">
<tr>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/1/1c/Al_sig5_1.gif" width="300" title="Minimization of Sigma5 grain boundary">
  <figcaption><strong>Figure 1.</strong> Movie showing the minimization of the grain boundary structure for an aluminum $\Sigma5(310)$ symmetric tilt grain boundary. </figcaption>
</figure>
</td>

<td width="290" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/e/e8/Al_sig5_2.gif" width="300" title="Minimization of Sigma5 grain boundary (with different starting position)">
    <figcaption><strong>Figure 2.</strong> Movie showing the minimization of the grain boundary structure for an aluminum $\Sigma5(310)$ symmetric tilt grain boundary (with a slightly different starting position, e.g., "delete_atoms overlap 1.5 lower upper"). </figcaption>
</figure>
</td>

</tr>
</table>



Figure 3 shows the image corresponding to the input script shown above. Notice how there are two atoms that are very close to each other at the boundary in Figure 3 (shown in red, colored by potential energy, i.e., high energy atoms). The final structures are shown in Figures 4 and 5 for the input script above and the modified input script, respectively. Interestingly, with this potential, the structure in Figure 4 is actually the lower energy grain boundary structure (567.57 mJ/m$^2$ in Figure 4 compared to 576.12 mJ/m$^2$ in Figure 5). The different structures for this one simple grain boundary may impart slightly different properties as well. Often, the convention is to use the minimum energy grain boundary structure for properties. However, some recent research has also explored metastable higher energy grain boundary structures and their impact on properties as well. 


<table width="800" border="0" cellpadding="5">
<tr>

<td width="250" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/f/fa/Sig5_0.jpg" width="300" title="Minimization of Sigma5 grain boundary">
  <figcaption><strong>Figure 3.</strong> Image showing the grain boundary structure prior to minimization. </figcaption>
</figure>
</td>

<td width="250" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/d/d3/Sig5_1.jpg" width="300" title="Minimization of Sigma5 grain boundary">
  <figcaption><strong>Figure 4.</strong> Image showing the grain boundary structure after minimization (overlap distance equals 0.35). </figcaption>
</figure>
</td>

<td width="250" align="center" valign="center">
<figure>
  <img src="https://icme.hpc.msstate.edu/mediawiki/images/9/91/00016.jpg" width="300" title="Minimization of Sigma5 grain boundary">
  <figcaption><strong>Figure 5.</strong> Image showing the grain boundary structure after minimization (overlap distance equals 1.50). </figcaption>
</figure>
</td>

</tr>
</table>



***
## FAQs 

</br>
<div class="alert alert-danger">
    <strong>Question 1</strong>: Why does the asymmetric grain boundary structure the lower energy?  
</div>

First, this result will be a function of the interatomic potential and the grain boundary configuration.  It just happens that the minimum energy $\Sigma5(310)$ grain boundary structure with this aluminum potential is asymmetric.  Others may not be so.  It should also be noted that the difference between the two structures is less than 10 mJ/m$^2$, so both may be observed in experiments at finite temperatures.

***
## Links

* [Click here to open Tutorial 1](LAMMPS-Tutorials-01.ipynb)
* [Click here to open Tutorial 2](LAMMPS-Tutorials-02.ipynb)
* [Click here to open Tutorial 3](LAMMPS-Tutorials-03.ipynb)
* [Click here to open Tutorial 4](LAMMPS-Tutorials-04.ipynb)
* [Click here to open the next tutorial](LAMMPS-Tutorials-06.ipynb)

***
## References 

1. S. Plimpton, "Fast Parallel Algorithms for Short-Range Molecular Dynamics," J. Comp. Phys., 117, 1-19 (1995). 
1. Tschopp, M. A., & McDowell, D.L. (2007). Structures and energies of Sigma3 asymmetric tilt grain boundaries in Cu and Al. Philosophical Magazine, 87, 3147-3173 (http://dx.doi.org/10.1080/14786430701455321). 
1. Tschopp, M. A., & McDowell, D.L. (2007). Asymmetric tilt grain boundary structure and energy in copper and aluminum. Philosophical Magazine, 87, 3871-3892 (http://dx.doi.org/10.1016/j.commatsci.2010.02.003).
1. Y. Mishin, D. Farkas, M.J. Mehl, and D.A. Papaconstantopoulos, "Interatomic potentials for monoatomic metals from experimental data and ab initio calculations," Phys. Rev. B 59, 3393 (1999). 
1. A. Stukowski, "Visualization and analysis of atomistic simulation data with OVITO – the Open Visualization Tool," Modelling Simul. Mater. Sci. Eng. 18 (2010) 015012.
1. J. Li, "AtomEye: an efficient atomistic configuration viewer," Modelling Simul. Mater. Sci. Eng. 11 (2003) 173. 